In [7]:
from yahooquery import Ticker
import requests
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
# !pip install selenium
import selenium

from selenium import webdriver

     ---------------------------------------- 6.3/6.3 MB 1.9 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [8]:
class StockDetail():

    api_key = 'ADLK0ZK57SB9BJKX'

    def __init__(self, symbol):

        self.symbol = symbol
    # returns a bunch of information like a description of the company, marketcap, 52 week lows and highs, EPS, Dividend, etc.
    def get_company_overview(self):

        url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol='+self.symbol+'&apikey='+self.api_key
        response = requests.get(url)
        response_json = response.json()

        if len(response_json) == 0:
            return "No data found."

        return response_json

    #retrieves current price of Stock
    def get_quote(self): 

        url = 'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol='+self.symbol+'&apikey='+self.api_key
        response = requests.get(url)
        response_json = response.json()

        if len(response_json) == 0:
            return "No data found."

        response_json = {x[4:].replace(' ','_'): v 
            for x, v in response_json['Global Quote'].items()}

        return response_json
    
    #monthly price data
    def get_monthly_adjusted(self):

        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol='+self.symbol+'&apikey='+self.api_key
        response = requests.get(url)
        response_json = response.json()

        if list(response_json.keys())[0] == 'Error Message':
            return "No data found."

        return response_json

    #live price data from yahooQuery
    def get_price(self):

        ticker = Ticker(self.symbol)
        raw_dict = ticker.price
        df = pd.DataFrame.from_dict(raw_dict)

        return df
    
    #returns earnings trend data
    def get_earnings_trend(self):

        ticker = Ticker(self.symbol)
        raw_dict = ticker.earnings_trend

        output_dict = {}
        output_dict['currentQtr'] = raw_dict[self.symbol]['trend'][0]['epsTrend']
        output_dict['nextQtr'] = raw_dict[self.symbol]['trend'][1]['epsTrend']
        output_dict['currentYr'] = raw_dict[self.symbol]['trend'][2]['epsTrend']
        output_dict['nextYr'] = raw_dict[self.symbol]['trend'][3]['epsTrend']

        return output_dict

    def get_option_chain(self, **kwargs):

        ticker = Ticker(self.symbol)
        df = ticker.option_chain

        if type(df) is str:
            if df == 'No option chain data found':  
                return "No data found."
                
        df.reset_index(inplace=True)
        df = df[df['optionType']=='puts']

        expireDates = df['expiration'].dt.strftime("%Y-%m-%d").unique()    

        df['mid'] = ( ( df['bid'] + df['ask'] ) / 2 ).round(2)
        df['timeToExpire'] = (df['expiration'] - pd.Timestamp.today()).round('1d').dt.days + 1
        df['multiplier'] = 365 / df['timeToExpire']
        df['return'] = ( df['multiplier'] * df['mid'] * 100 ) / ( df['strike'] * 100 )
        df.drop(['optionType', 'contractSymbol', 'currency', 'contractSize', 'lastTradeDate', 'impliedVolatility', 'multiplier'], axis=1, inplace=True)
        df['change'] = df['change'].round(2)
        df['percentChange'] = df['percentChange'].round(2)
        df['return'] = (df['return'] * 100).round(2)

        # Set filters
        if kwargs['expiration'] != 'all' and kwargs['expiration'] != None:
            df = df[ df['expiration'] == kwargs['expiration'] ]

        if kwargs['strikeMin'] != '' and kwargs['strikeMin'] != None:
            df = df[ df['strike'] >= int(kwargs['strikeMin']) ]

        if kwargs['strikeMax'] != '' and kwargs['strikeMax'] != None:
            df = df[ df['strike'] <= int(kwargs['strikeMax']) ]

        if kwargs['returnMin'] != '' and kwargs['returnMin'] != None:
            df = df[ df['return'] >= int(kwargs['returnMin']) ]

        if kwargs['returnMax'] != '' and kwargs['returnMax'] != None:
            df = df[ df['return'] <= int(kwargs['returnMax']) ]

        if kwargs['timeToExpireMin'] != '' and kwargs['timeToExpireMin'] != None:
            df = df[ df['timeToExpire'] >= int(kwargs['timeToExpireMin']) ]

        if kwargs['timeToExpireMax'] != '' and kwargs['timeToExpireMax'] != None:
            df = df[ df['timeToExpire'] >= int(kwargs['timeToExpireMax']) ]

        # Format Columns
        df['expiration'] = df['expiration'].dt.strftime("%Y-%m-%d")

        # Rename Columns
        column_rename = {
            'percentChange': '% Change',
            'openInterest': 'open Int'
        }
        df.rename(columns=column_rename, inplace=True)

        results = {}
        results['columns'] = df.columns.values
        results['expiration'] = expireDates
        results['data'] = df

        return results


In [9]:
#YAHOO QUERY
#returns earnings trend data 
# returns earnings trend for future dates
def get_earnings_trend(symbol):

    ticker = Ticker(symbol)
    raw_dict = ticker.earnings_trend
    display(raw_dict)
    output_dict = {}
    output_dict['currentQtr'] = raw_dict[symbol]['trend'][0]['epsTrend']
    output_dict['nextQtr'] = raw_dict[symbol]['trend'][1]['epsTrend']
    output_dict['currentYr'] = raw_dict[symbol]['trend'][2]['epsTrend']
    output_dict['nextYr'] = raw_dict[symbol]['trend'][3]['epsTrend']

    return output_dict

get_earnings_trend('AAPL')

{'AAPL': {'trend': [{'maxAge': 1,
    'period': '0q',
    'endDate': '2023-03-31',
    'growth': -0.059,
    'earningsEstimate': {'avg': 1.43,
     'low': 1.37,
     'high': 1.56,
     'yearAgoEps': 1.52,
     'numberOfAnalysts': 26,
     'growth': -0.059},
    'revenueEstimate': {'avg': 93192800000,
     'low': 91805900000,
     'high': 98836000000,
     'numberOfAnalysts': 24,
     'yearAgoRevenue': 97278000000,
     'growth': -0.042},
    'epsTrend': {'current': 1.43,
     '7daysAgo': 1.49,
     '30daysAgo': 1.5,
     '60daysAgo': 1.49,
     '90daysAgo': 1.48},
    'epsRevisions': {'upLast7days': 3,
     'upLast30days': 8,
     'downLast30days': 14,
     'downLast90days': {}}},
   {'maxAge': 1,
    'period': '+1q',
    'endDate': '2023-06-30',
    'growth': 0.033,
    'earningsEstimate': {'avg': 1.24,
     'low': 1.02,
     'high': 1.35,
     'yearAgoEps': 1.2,
     'numberOfAnalysts': 26,
     'growth': 0.033},
    'revenueEstimate': {'avg': 85587200000,
     'low': 81317700000,
  

{'currentQtr': {'current': 1.43,
  '7daysAgo': 1.49,
  '30daysAgo': 1.5,
  '60daysAgo': 1.49,
  '90daysAgo': 1.48},
 'nextQtr': {'current': 1.24,
  '7daysAgo': 1.24,
  '30daysAgo': 1.26,
  '60daysAgo': 1.26,
  '90daysAgo': 1.26},
 'currentYr': {'current': 6.04,
  '7daysAgo': 6.15,
  '30daysAgo': 6.2,
  '60daysAgo': 6.24,
  '90daysAgo': 6.26},
 'nextYr': {'current': 6.61,
  '7daysAgo': 6.69,
  '30daysAgo': 6.75,
  '60daysAgo': 6.8,
  '90daysAgo': 6.81}}

# YAHOO QUERY START OF SELENIUM Premium HISTORICAL EARNINGS

##### documentation for yahoo query premium
https://yahooquery.dpguthrie.com/guide/ticker/premium/#p_get_financial_data

In [10]:
# !pip install yahooquery[premium]
# !pip uninstall selenium
# !pip install selenium==3.5.0
import selenium
from selenium import webdriver
from yahooquery import Ticker

yf_username = 'qcappartners@yahoo.com'

yf_password = 'Mcci2022!!'
# ticker = Ticker('aapl', username= yf_username,
#     password= yf_password)
symbols = ['aapl']

faang = Ticker(symbols, asynchronous=True)
faang.login(username=yf_username, password=yf_password)


In [28]:
# drilldown = Ticker(symbols, username=yf_username, password=yf_password).p_value_analyzer_drilldown
# display(drilldown)
tickers = ['aapl', 'nflx']
# 'DilutedEPS', 'BasicEPS', 
fin_types = ['DilutedEPS', 'BasicEPS']
financial_data = Ticker(tickers, username=yf_username, password=yf_password).p_get_financial_data(types=fin_types, trailing=False)
display(financial_data)


# aapl.p_value_analyzer_drilldown
# Ticker('aapl').p_value_analyzer_drilldown

,asOfDate,periodType,currencyCode,BasicEPS,DilutedEPS
symbol,,,,,
aapl,1985-09-30,12M,USD,0.004464,0.004464
aapl,1986-09-30,12M,USD,0.010714,0.010714
aapl,1987-09-30,12M,USD,0.014821,0.014821
aapl,1988-09-30,12M,USD,0.027500,0.027500
aapl,1989-09-30,12M,USD,0.031607,0.031607
aapl,1990-09-30,12M,USD,0.033750,0.033750
aapl,1991-09-30,12M,USD,0.023036,0.023036
aapl,1992-09-30,12M,USD,0.038750,0.038750
aapl,1993-09-30,12M,USD,0.006607,0.006607


In [13]:
global AlphaVantageKey
#basic key AlphaVantageKey = 'XEQ9AFU8KM035KMG'
#premium api key
AlphaVantageKey = 'ZTB6U564ILR50HU3'


def fairValue_hist(ticker):
    global AlphaVantageKey
    global getFairValueTime    
    yq_ticker = Ticker(ticker)
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+AlphaVantageKey
    r = requests.get(url)
    data = r.json()
    # print(type(data['annualEarnings']))
    # display(data['fannualEarnings'])
    df = pd.DataFrame(data['annualEarnings'])
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    filtered_values = np.where((df['fiscalDateEnding'] > '2017-01-01') & (df['fiscalDateEnding'] < '2020-01-01'))
    # display(filtered_values)
    eps1 = df.loc[filtered_values]
#     display(eps1)
    # print(filtered_values)

    testPrice = []
    reportedEPS_cols = []
    pe_cols = []
    testPrice_cols = []

    for index, row in eps1.iterrows():
        #retrieving testPrice from Past #change to 30-60 days average
        testPriceDateStart = row['fiscalDateEnding'] + relativedelta(months=+3)
        testPriceDateEnd = row['fiscalDateEnding'] + relativedelta(months=+4)
        df = yq_ticker.history(period='6y', interval='1d')
        priceHistory = pd.DataFrame(df)
        priceHistory = priceHistory.reset_index()
        priceHistory['date']= pd.to_datetime(priceHistory['date'])
        mask = (priceHistory['date'] > testPriceDateStart) & (priceHistory['date'] < testPriceDateEnd)
        avg_price = priceHistory.loc[mask]
        avgTestPrice = avg_price['close'].mean()
        testPrice.append(avgTestPrice)
        currentYear = row['fiscalDateEnding'].strftime("%Y")

        #making all the columns
        reportedEPS_cols.append('reportedEPS' + currentYear)
        pe_cols.append('p_e'+currentYear)
        testPrice_cols.append('avgPrice' + currentYear)
        testdf = pd.DataFrame(testPrice)
#         print('testdf')
#         display(testdf)

    #building vertical dataframe which ends up getting transposed
    eps1.reset_index(drop=True)
    eps1['testPrice'] = testdf.values
    eps1['reportedEPS'] = eps1['reportedEPS'].astype(float)
    eps1['p_e'] = eps1['testPrice']/eps1['reportedEPS']
    eps1['avgp_e'] = eps1['p_e'].mean()
    #current yr estimate
    currentYrEstimate = StockDetail(ticker).get_earnings_trend()['currentYr']['current']
    eps1['currentYrEstimate'] = currentYrEstimate
    eps1['FairValue'] = eps1['avgp_e'] * eps1['currentYrEstimate']
    
    #bulding long data lists which is what we actually use for fv_df
    data = eps1['reportedEPS'].to_list()
    data.extend(eps1['testPrice'].to_list())
    data.extend(eps1['p_e'].to_list())
    data.append(eps1['currentYrEstimate'].values[0])
    data.append(eps1['FairValue'].values[0])

    #creating final long data row from lists
    fv_df_cols = reportedEPS_cols + testPrice_cols + pe_cols + ['currentYrEstimate', 'FairValue']
#     print(fv_df_cols)
    fv_df = pd.DataFrame(data = [data], columns = fv_df_cols)

    # executionTime = (time.time() - startTime)
    # getFairValueTime = getFairValueTime + executionTime

    return fv_df

fairValue_hist('AAPL')

,reportedEPS2019,reportedEPS2018,reportedEPS2017,avgPrice2019,avgPrice2018,avgPrice2017,p_e2019,p_e2018,p_e2017,currentYrEstimate,FairValue
0,2.98,2.97,2.3,77.631375,38.29425,43.680395,26.050797,12.893687,18.991476,6.15,118.768718
